In [1]:
pip install ortools

     |████████████████████████████████| 14.0MB 319kB/s 
     |████████████████████████████████| 133kB 51.3MB/s 
     |████████████████████████████████| 1.0MB 42.4MB/s 
ERROR: tensorflow-metadata 0.27.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [7]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

In [8]:
matrix_1 = [
  [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
  [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
  [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
  [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
  [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
  [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
  [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
  [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
  [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
  [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
  [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
  [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
  [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
]
dist_function_1 = lambda a, b : matrix_1[a][b]
locations_1 = list(range(len(matrix_1)))
capacities_1 = [100, 100, 100, 100]
demands_1 = [0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

matrix_2 = [[0, 1, 1], [1, 0, 1], [1, 1, 0]]
dist_function_2 = lambda a, b : matrix_2[a][b]
locations_2 = list(range(len(matrix_2)))
capacities_2 = [(10, 0), (0, 10)]
demands_2 = [(0, 0), (1, 0), (0, 1)]

# TSP

In [4]:
def tsp(dist_function, locations):  # function takes loc 1 and loc 2 and returns dist between

  def create_dist_matrix(locations):
    matrix = np.zeros((len(locations), len(locations))).tolist()
    for l1 in range(len(matrix)):
      for l2 in range(len(matrix)):
        matrix[l1][l2] = dist_function(locations[l1], locations[l2])
    return matrix
        
  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  dist_matrix = create_dist_matrix(locations)
            
  manager = pywrapcp.RoutingIndexManager(len(dist_matrix), 1, 0)

  routing = pywrapcp.RoutingModel(manager)
  transit_callback_index = routing.RegisterTransitCallback(distance_callback)

  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  visited = []
  index = routing.Start(0)
  while not routing.IsEnd(index):
    visited.append(manager.IndexToNode(index))
    index = solution.Value(routing.NextVar(index))
  visited.append(manager.IndexToNode(index))
  
  return visited, solution.ObjectiveValue()

In [5]:
tsp(dist_function_1, locations_1)

([0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], 7293)

# VRP

In [75]:
def vrp(dist_matrix, locations, no_vehicles, depot=0, time_limit=-1):

  assert len(dist_matrix) == len(dist_matrix[0]) == len(locations)

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  manager = pywrapcp.RoutingIndexManager(len(locations), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, 0, int(1e9), True, "Distance")
  distance_dimension = routing.GetDimensionOrDie("Distance")
  distance_dimension.SetGlobalSpanCostCoefficient(100)
  
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)
    
  return visited, max_route_distance

In [76]:
vrp(matrix_1, locations_1, 2)

([[0, 10, 5, 11, 1, 4, 2, 7, 0], [0, 3, 6, 8, 12, 9, 0]], 5728)

# Capacity constraints

In [79]:
def cvrp(dist_matrix, locations, no_vehicles, demands, capacities, depot=0, time_limit=-1):

  assert len(dist_matrix) == len(dist_matrix[0]) == len(locations) == len(demands)
  assert len(capacities) == no_vehicles

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return demands[from_node]

  manager = pywrapcp.RoutingIndexManager(len(locations), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
  routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, capacities, True, 'Capacity')
  
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
  search_parameters.time_limit.FromSeconds(1)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  loads = [0 for __ in range(len(capacities))]
  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          loads[vehicle_id] += demands[manager.IndexToNode(index)]
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)

  return visited, max_route_distance, loads

In [80]:
cvrp(matrix_1, locations_1, len(capacities_1), demands_1, capacities_1)

([[0, 0], [0, 0], [0, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], [0, 2, 7, 0]],
 6892,
 [0, 0, 100, 20])

# Pickups and Deliveries

In [82]:
matrix = [[0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
          [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
          [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
          [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
          [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
          [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
          [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
          [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
          [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
          [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
          [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
          [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
          [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
          [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
          [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
          [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
          [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0]]
routes = [[1, 6], [2, 10], [4, 3], [5, 9], [7, 8], [15, 11], [13, 12], [16, 14]]

In [88]:
def picks_dels(dist_matrix, locations, no_vehicles, routes, depot=0, time_limit=-1):

  assert len(dist_matrix) == len(dist_matrix[0]) == len(locations)

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  manager = pywrapcp.RoutingIndexManager(len(dist_matrix), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, 0, int(1e9), True, "Distance")
  distance_dimension = routing.GetDimensionOrDie("Distance")
  distance_dimension.SetGlobalSpanCostCoefficient(100)

  for request in routes:
    pickup_index = manager.NodeToIndex(request[0])
    delivery_index = manager.NodeToIndex(request[1])
    routing.AddPickupAndDelivery(pickup_index, delivery_index)
    routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
    routing.solver().Add(distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index))

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  routes = [[] for __ in range(no_vehicles)]
  distances = []
  for vehicle_id in range(no_vehicles):
    index = routing.Start(vehicle_id)
    route_distance = 0
    while not routing.IsEnd(index):
      routes[vehicle_id].append(manager.IndexToNode(index))
      previous_index = index
      index = solution.Value(routing.NextVar(index))
      route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
    routes[vehicle_id].append(manager.IndexToNode(index))
    distances.append(route_distance)
  
  return routes, distances

In [89]:
picks_dels(matrix, list(range(len(matrix))), 4, routes)

([[0, 5, 2, 10, 16, 14, 9, 0],
  [0, 7, 1, 6, 8, 0],
  [0, 13, 15, 11, 12, 0],
  [0, 4, 3, 0]],
 [2192, 1780, 1552, 1392])

# Time windows (WIP)

In [ ]:
def pickups_deliveries(dist_matrix, routes, no_vehicles, depot=0):

    manager = pywrapcp.RoutingIndexManager(len(dist_matrix), no_vehicles, depot)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data[dist_matrix][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    routing.AddDimension(transit_callback_index, 0, 3000, True, "Distance")
    distance_dimension = routing.GetDimensionOrDie("Distance")
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
        routing.solver().Add(distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index))

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    solution = routing.SolveWithParameters(search_parameters)

    if not solution:
        raise ValueError("NO SOLUTION :(")

    vehicles_routes = [[] for i in range(no_vehicles)]
    for vehicle_id in range(no_vehicles):
    


    return 


    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))


In [ ]:

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    data['pickups_deliveries'] = [
        [1, 6],
        [2, 10],
        [4, 3],
        [5, 9],
        [7, 8],
        [15, 11],
        [13, 12],
        [16, 14],
    ]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data


# tsp/vrp/cvrp combined general function (Not working :[  )

In [39]:
def routing(dist_function, locations, no_vehicles=1, capacities=[], demands=[], time_windows=[], max_travel_dist=int(1e9), time_limit=-1):

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  if capacities == []:  # if no capacity entered assume infinite capacity
    capacities = [(int(1e9),) for __ in range(no_vehicles)]
  if demands == []:  # if no demand entered assume 1 demand
    demands = [(1,) for __ in range(len(locations))]

  if type(demands[0]) != tuple:
    capacities = [(i,) for i in capacities]
    demands = [(i,) for i in demands]

  if len(time_windows) == 0:
    time_windows = [(0, max_travel_dist) for __ in range(len(locations))]

  assert len(capacities) == no_vehicles
  assert len(demands) == len(locations) == len(time_windows)

  def create_dist_matrix(locations):
    matrix = np.zeros((len(locations), len(locations))).tolist()
    for l1 in range(len(matrix)):
      for l2 in range(len(matrix)):
        matrix[l1][l2] = dist_function(locations[l1], locations[l2])
    return matrix
        
  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  def demand_callback(from_index, idx):  # callback for material: idx
    from_node = manager.IndexToNode(from_index)
    return demands[from_node][idx]

  def add(a, b):
    return [a[i]+b[i] for i in range(len(a))]

  dist_matrix = create_dist_matrix(locations)
            
  manager = pywrapcp.RoutingIndexManager(len(dist_matrix), no_vehicles, 0)
  routing = pywrapcp.RoutingModel(manager)
  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, 0, max_travel_dist, True, 'Distance')  # For multiple vehicles
  distance_dimension = routing.GetDimensionOrDie('Distance')
  distance_dimension.SetGlobalSpanCostCoefficient(100)

  for i in range(len(capacities[0])):  # add dimension for each capacity
    demand_callback_index = routing.RegisterUnaryTransitCallback(lambda x : demand_callback(x, i))
    routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, [j[i] for j in capacities], True, f'Capacity_{i}')

  routing.AddDimension(transit_callback_index, int(1e9), int(1e9), False, "Time")  # For time constraints
  time_dimension = routing.GetDimensionOrDie("Time")
  for location_idx, time_window in enumerate(time_windows):
    if location_idx == 0:
      continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
  for vehicle_id in range(no_vehicles):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(time_windows[0][0], time_windows[0][1])
  for i in range(no_vehicles):
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))
    
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  loads = [[0 for __ in range(len(capacities[0]))] for __ in range(len(capacities))]
  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          loads[vehicle_id] = add(loads[vehicle_id], demands[manager.IndexToNode(index)])
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)

  return visited, max_route_distance, loads  # route (vehicles, location idx), cost

In [40]:
def tsp(dist_function, locations):
  x = routing(dist_function, locations)
  return x[0][0], x[1]

In [41]:
def vrp(dist_function, locations, no_vehicles):
  x = routing(dist_function, locations, no_vehicles=no_vehicles)
  return x[:2]

In [42]:
def cvrp(dist_function, locations, no_vehicles, capacities, demands):
  x = routing(dist_function, locations, no_vehicles=no_vehicles, capacities=capacities, demands=demands)
  if len(x[2][0]) == 1:
    return x[0], x[1], [i[0] for i in x[2]]
  return x[0], x[1], x[2]

In [43]:
tsp(dist_function_1, locations_1)

([0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], 7293)

In [44]:
vrp(dist_function_1, locations_1, 2)  # not working because have multiple things to maximise

SystemError: ignored

In [32]:
cvrp(dist_function_1, locations_1, len(capacities_1), capacities_1, demands_1)

([[0, 12, 8, 2, 0], [0, 9, 4, 1, 11, 5, 0], [0, 10, 0], [0, 3, 6, 7, 0]],
 5143,
 [30, 50, 10, 30])

In [13]:
cvrp(dist_function_2, locations_2, len(capacities_2), capacities_2, demands_2)  # WHY DOESNT THIS WORK ??!!!

ValueError: ignored

# Modular thing (Not working :[  )

In [69]:
def vrp_base(dist_matrix, locations, no_vehicles, update_routing=lambda x : None, depot=0, time_limit=-1):

  assert len(dist_matrix) == len(dist_matrix[0]) == len(locations)

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  manager = pywrapcp.RoutingIndexManager(len(locations), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, 0, int(1e9), True, "Distance")
  distance_dimension = routing.GetDimensionOrDie("Distance")
  distance_dimension.SetGlobalSpanCostCoefficient(100)

  update_routing(routing)

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  return solution, routing, manager

In [40]:
def tsp(dist_matrix, locations, depot=0, time_limit=-1):
  solution, routing, manager = vrp_base(dist_matrix, locations, 1, depot=depot, time_limit=time_limit)

  visited = []
  index = routing.Start(0)
  route_distance = 0
  while not routing.IsEnd(index):
    visited.append(manager.IndexToNode(index))
    previous_index = index
    index = solution.Value(routing.NextVar(index))
    route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
  visited.append(manager.IndexToNode(index))
  
  return visited, route_distance

In [61]:
def vrp(dist_matrix, locations, no_vehicles, depot=0, time_limit=-1):

  solution, routing, manager = vrp_base(dist_matrix, locations, no_vehicles, depot=depot, time_limit=time_limit)

  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)
    
  return visited, max_route_distance

In [70]:
def cvrp(dist_matrix, locations, no_vehicles, demands, capacities, depot=0, time_limit=-1):

  if type(demands[0]) != tuple:
    demands = [(i,) for i in demands]
    capacities = [(i,) for i in capacities]

  assert len(demands) == len(locations)
  assert len(capacities) == no_vehicles

  def demand_callback(from_index, cargo_type):
    from_node = manager.IndexToNode(from_index)
    return demands[cargo_type][from_node]

  def update_routing(routing):
    for i in range(len(demands[0])):
      callback = lambda x : demand_callback(x, i)
      demand_callback_index = routing.RegisterUnaryTransitCallback(callback)
      routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, [j[i] for j in capacities], True, f'Capacity_{i}')

  solution, routing, manager = vrp_base(dist_matrix, locations, no_vehicles, update_routing=update_routing, depot=depot, time_limit=depot)

  loads = [[0 for __ in range(len(capacities[0]))] for __ in range(len(capacities))]
  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          loads[vehicle_id] = add(loads[vehicle_id], demands[manager.IndexToNode(index)])
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)

  return visited, max_route_distance, loads

In [ ]:
def vrptw():
  pass

In [57]:
tsp(matrix_1, locations_1)

([0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], 7293)

In [58]:
vrp(matrix_1, locations_1, 2)

([[0, 10, 5, 11, 1, 4, 2, 7, 0], [0, 3, 6, 8, 12, 9, 0]], 5728)

In [71]:
cvrp(matrix_1, locations_1, len(capacities_1), demands_1, capacities_1)

SystemError: ignored

In [45]:
cvrp(dist_function_2, locations_2, len(capacities_2), capacities_2, demands_2)  # WHY DOESNT THIS WORK ??!!!

AssertionError: ignored